In [ ]:
!pip install selenium
!pip install webdriver-manager

In [ ]:
# chrome 실행
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# Selenium에서 웹 요소를 찾기 위한 다양한 방법을 제공하는 By 클래스를 임포트하는 구문
from selenium.webdriver.common.by import By
  # driver.find_element(By.XPATH, "//button[@type='submit']")

# WebDriverWait는 특정 조건이 만족될 때까지 대기하는 기능을 제공
from selenium.webdriver.support.ui import WebDriverWait
  # wait = WebDriverWait(driver, timeout=10, poll_frequency=1)

# EC는 WebDriverWait와 함께 사용되는 미리 정의된 조건들의 집합
from selenium.webdriver.support import expected_conditions as EC
  # element = wait.until(EC.element_to_be_clickable((By.ID, "submit")))

# Chrome 옵션 설정
chrome_options = Options()
# chrome_options.add_argument("--headless")  # GUI 없이 실행 (선택 사항)
chrome_options.add_argument("--no-sandbox") # 권한 문제 방지 (Linux 환경에서 필요할 수 있음)
chrome_options.add_argument("--disable-dev-shm-usage") # 메모리 문제 방지 (Linux 환경에서 필요할 수 있음)
# ChromeDriverManager를 사용하여 ChromeDriver 자동 설치 및 설정
service = Service(ChromeDriverManager().install())
# WebDriver 객체 생성
driver = webdriver.Chrome(service=service, options=chrome_options)


In [ ]:
# 특정 변수 모음
url_nexon = "https://fconline.nexon.com/main/index"

xpath_login = "/html/body/div[1]/div[1]/div[2]/div[4]/a"
xpath_id = "/html/body/div[2]/div[2]/div/div[1]/fieldset/div[1]/input"
xpath_pw = "/html/body/div[2]/div[2]/div/div[1]/fieldset/div[2]/input"
xpath_id_btn ="/html/body/div[2]/div[2]/div/div[1]/fieldset/div[4]/button"
xpath_start_btn ="/html/body/div[2]/header/a[3]"

In [ ]:
# xpath 있는지 체크
def check_xpath_exists(driver, xpath, timeout=10):
    try:
        element = WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.XPATH, xpath))
        )
        return True
    except TimeoutException:
        return False

# 사용 예시
xpath_to_check = xpath_login
if check_xpath_exists(driver, xpath_to_check):
    print(f"XPath '{xpath_to_check}'가 존재합니다.")
else:
    print(f"XPath '{xpath_to_check}'가 존재하지 않습니다.")

In [ ]:
import time

# login 처리
def element_push(driver, xpath, input_data=None, max_retries=3):
    for attempt in range(max_retries):
        try:
            # 최대 10초간 페이지 로딩 / 특정 xpath 검색될때 까지 
            element = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, xpath))
            )
            element.click()
            
            # 클릭 후 잠시 대기
            time.sleep(0.5)
            
            # input_data가 제공된 경우에만 send_keys 실행
            if input_data is not None:
                element.send_keys(input_data)
            
            return True  # 성공적으로 처리됨
        
        except TimeoutException:
            print(f"요소를 찾는 데 시간이 초과되었습니다. XPath: {xpath}")
        except ElementClickInterceptedException:
            print(f"요소를 클릭하려 했으나 다른 요소에 의해 가려져 있습니다. XPath: {xpath}")
        except StaleElementReferenceException:
            print(f"요소가 더 이상 DOM에 존재하지 않습니다. 페이지가 변경되었을 수 있습니다. XPath: {xpath}")
        except Exception as e:
            print(f"요소를 처리하는 중 예상치 못한 오류가 발생했습니다: {e}")
        
        if attempt < max_retries - 1:
            print(f"재시도 중... ({attempt + 1}/{max_retries})")
            time.sleep(1)  # 재시도 전 잠시 대기
        else:
            print("최대 재시도 횟수를 초과했습니다.")
    
    return False  # 모든 시도 실패

In [ ]:
# 웹사이트 접속
driver.get(url_nexon)
# login 버튼 클릭
element_push(driver,xpath_login)
# id pw 입력 후 로그인
element_push(driver,xpath_id,"amj90")
element_push(driver,xpath_pw,"vlvk123123")
element_push(driver,xpath_id_btn)
# login 체크크

In [ ]:
# 특정 xpath가 있는지 확인하고 있으면 그 데이터를 출력하는 코드
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

def check_and_get_element_value(driver, xpath, attribute=None, timeout=10):
    try:
        element = WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.XPATH, xpath))
        )
        
        print(f"Element with XPath '{xpath}' exists.")
        
        if attribute:
            value = element.get_attribute(attribute)
            print(f"Attribute '{attribute}' value: {value}")
        else:
            value = element.text
            print(f"Element text: {value}")
        
        return value
    
    except TimeoutException:
        print(f"Element with XPath '{xpath}' was not found within {timeout} seconds.")
        return None
    except NoSuchElementException:
        print(f"Element with XPath '{xpath}' does not exist.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# 사용 예시
xpath = xpath_login
result = check_and_get_element_value(driver, xpath)

# 특정 속성(예: href) 값을 가져오고 싶다면:
# href_result = check_and_get_element_value(driver, xpath, attribute="href")


Element with XPath '/html/body/div[1]/div[1]/div[2]/div[4]/a' exists.
Element text: 
Element with XPath '/html/body/div[1]/div[1]/div[2]/div[4]/a' exists.
Attribute 'href' value: https://fconline.nexon.com/main/index#


In [18]:
try:
    # XPath 지정 (원하는 요소의 XPath로 변경하세요)
    xpath = xpath_login

    # 요소가 로드될 때까지 대기 (최대 10초)
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, xpath))
    )

    # 요소의 텍스트 가져오기
    text = element.text
    print("추출된 텍스트:", text)

except Exception as e:
    print("오류 발생:", str(e))

추출된 텍스트: 


In [20]:
xpath = xpath_login

# 요소가 로드될 때까지 대기 (최대 10초)
element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, xpath))
)

# 요소의 텍스트 가져오기
text = element.text
print("추출된 텍스트:", text)

추출된 텍스트: 
